In [1]:
!nvidia-smi

Thu Nov 21 04:54:17 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.104      Driver Version: 410.104      CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0    34W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import numpy as np
import time

In [3]:
print(tf.__version__)
print(tf.test.is_gpu_available())

2.0.0
True


In [4]:
# Enable XLA
tf.config.optimizer.set_jit(True)

# Enable AMP
tf.keras.mixed_precision.experimental.set_policy('mixed_float16')

In [5]:
X_train = np.load('Data/X_train.npy', allow_pickle=True)
y_train = np.load('Data/y_train.npy', allow_pickle=True)

In [6]:
X_train.shape, y_train.shape

((6252, 224, 224, 3), (6252,))

In [7]:
X_train.dtype

dtype('float64')

In [8]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

In [9]:
IMG_SIZE = (224, 224)

@tf.function
def format_train_example(image, label):
    image = tf.cast(image, tf.float32)
    image = (image/127.5) - 1
    image = tf.image.resize(image, IMG_SIZE)
    # return images as float16
    image = tf.cast(image, tf.float16)
    return image, tf.one_hot(label, 5)

In [10]:
BATCH_SIZE = 80
AUTOTUNE = tf.data.experimental.AUTOTUNE
PREFETCH_COUNT = 8

train_dataset = train_dataset.shuffle(1024)
train_dataset = train_dataset.repeat()
train_dataset = train_dataset.map(format_train_example,
                                  num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(PREFETCH_COUNT)

In [11]:
def create_model(img_size=(224,224), num_class=5, train_base=True):
    # Accept float16 image inputs
    input_layer = Input(shape=(img_size[0],img_size[1],3), dtype=tf.float16)
    base = ResNet50(input_tensor=input_layer,
                    include_top=False,
                    weights="imagenet")
    base.trainable = train_base
    x = base.output
    x = GlobalAveragePooling2D()(x)
    # softmax only accepts float32 - need to manually cast (likely a bug)
    preds = Dense(num_class, activation="softmax", dtype=tf.float32)(x)
    return Model(inputs=input_layer, outputs=preds)

In [12]:
model = create_model()
opt = Adam()
model.compile(loss="categorical_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])

In [13]:
# Kickstart model training
start = time.time()
model.fit(train_dataset, 
         steps_per_epoch=X_train.shape[0]//BATCH_SIZE,
         epochs=5)
print("It took {} seconds".format(time.time() - start))

Train for 78 steps
Epoch 1/5
78/78 [==============================] - 69s 886ms/step - loss: 0.6534 - accuracy: 0.5939
Epoch 2/5
78/78 [==============================] - 19s 241ms/step - loss: 0.8044 - accuracy: 0.5321
Epoch 3/5
78/78 [==============================] - 18s 235ms/step - loss: 0.9210 - accuracy: 0.4814
Epoch 4/5
78/78 [==============================] - 18s 235ms/step - loss: 0.9866 - accuracy: 0.4279
Epoch 5/5
78/78 [==============================] - 18s 235ms/step - loss: 1.0228 - accuracy: 0.4075
It took 143.18667888641357 seconds
